In [13]:
import os
from glob import glob
from keras.models import load_model
import h5py

In [14]:
all_models = glob(os.path.join('*', '*', '*.h5'))
node_models = [x for x in all_models if 'nodule' in x]
print(all_models[0], node_models)

scoring_code/ensemble 2 models/model_des_v38_mse_64_finetune_04.h5 ['scoring_code/nodule models/model_clf_v1_64_finetune_04.h5', 'scoring_code/nodule models/model_LUNA_64_v29_14.h5']


In [13]:
load_model(node_models[1], 
           compile=True, 
           custom_objects=dict(L1L2Regularizer = lambda x: x))

/Users/mader/anaconda/lib/python3.5/site-packages/keras/engine/topology.py:1242: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(dtype="float32", name="input_2", batch_input_shape=[None, 1, ..., sparse=False)`
  return cls(**config)
/Users/mader/anaconda/lib/python3.5/site-packages/keras/engine/topology.py:1242: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(padding="same", name="convolution3d_6", strides=[1, 1, 1], trainable=True, kernel_size=(3, 3, 3), activity_regularizer=None, data_format="channels_first", bias_regularizer=None, kernel_regularizer={'name': '..., activation="linear", filters=8, kernel_initializer="glorot_uniform", bias_constraint=None, use_bias=True, kernel_constraint=None)`
  return cls(**config)


ValueError: Improper config format: {'name': 'L1L2Regularizer', 'l2': 9.999999747378752e-05, 'l1': 0.0}

In [28]:
with h5py.File(node_models[1], 'r') as f:
    print(list(f.attrs.items()))
    for k,v in f.items():
        print(k,list(v.attrs))
    layer_info = list(f['model_weights'].attrs.items())
    for k,v in f['model_weights'].items():
        print(k,v)
    

[('keras_version', b'1.2.0'), ('model_config', b'{"class_name": "Model", "config": {"layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 1, 64, 64, 64], "input_dtype": "float32", "sparse": false, "name": "input_2"}, "inbound_nodes": [], "name": "input_2"}, {"class_name": "Convolution3D", "config": {"W_constraint": null, "b_constraint": null, "name": "convolution3d_6", "kernel_dim2": 3, "kernel_dim1": 3, "activation": "linear", "trainable": true, "dim_ordering": "th", "kernel_dim3": 3, "subsample": [1, 1, 1], "init": "glorot_uniform", "bias": true, "nb_filter": 8, "b_regularizer": null, "W_regularizer": {"l2": 9.999999747378752e-05, "name": "L1L2Regularizer", "l1": 0.0}, "activity_regularizer": null, "border_mode": "same"}, "inbound_nodes": [[["input_2", 0, 0]]], "name": "convolution3d_6"}, {"class_name": "BatchNormalization", "config": {"gamma_regularizer": null, "name": "batchnormalization_13", "epsilon": 0.001, "trainable": true, "mode": 0, "beta_regularizer

In [23]:
layer_info

[('layer_names',
  array([b'input_2', b'convolution3d_6', b'batchnormalization_13',
         b'leakyrelu_12', b'maxpooling3d_5', b'averagepooling3d_5',
         b'merge_5', b'convolution3d_7', b'batchnormalization_14',
         b'leakyrelu_13', b'maxpooling3d_6', b'averagepooling3d_6',
         b'merge_6', b'convolution3d_8', b'batchnormalization_15',
         b'leakyrelu_14', b'maxpooling3d_7', b'averagepooling3d_7',
         b'merge_7', b'convolution3d_9', b'batchnormalization_16',
         b'leakyrelu_15', b'maxpooling3d_8', b'averagepooling3d_8',
         b'merge_8', b'convolution3d_10', b'batchnormalization_17',
         b'leakyrelu_16', b'globalmaxpooling3d_2', b'batchnormalization_18',
         b'dense_7', b'dense_10', b'dense_11', b'dense_12', b'dense_8',
         b'batchnormalization_19', b'batchnormalization_22',
         b'batchnormalization_23', b'batchnormalization_24',
         b'batchnormalization_20', b'leakyrelu_17', b'leakyrelu_20',
         b'leakyrelu_21', b'leakyre

In [7]:
import keras.backend as K
K.set_image_dim_ordering('th')

In [1]:
import os, sys
sys.path.append(os.path.join('training_code','FLung_nodule_models'))
from build_luna_model_v29 import build_model

Using TensorFlow backend.


In [9]:
n_model = build_model((1, 64, 64, 64))

training_code/FLung_nodule_models/build_luna_model_v29.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(8, (3, 3, 3), kernel_regularizer=<keras.reg..., padding="same")`
  x1 = Convolution3D(num_filters,3,3,3,border_mode='same',W_regularizer=l2(1e-4))(x_input)
training_code/FLung_nodule_models/build_luna_model_v29.py:80: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x1_merged = merge([x1, x1_ident],mode='concat', concat_axis=1)
/Users/mader/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
training_code/FLung_nodule_models/build_luna_model_v29.py:44: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(24, (3, 3, 3), kernel_regularizer=<keras.reg..., 

compiling model


In [15]:
n_model.load_weights(node_models[1])

In [16]:
n_model.save('trained_node_model.h5')

In [26]:
for i in n_model.layers:
    print(i.name,i.input_shape[1:],'->',i.output_shape[1:], 'Strides:', i.get_config().get('strides'), 
          'KernelSize:', i.get_config().get('kernel_size'), i.get_config().get('activation'))

input_4 (1, 64, 64, 64) -> (1, 64, 64, 64) Strides: None KernelSize: None None
conv3d_8 (1, 64, 64, 64) -> (8, 64, 64, 64) Strides: (1, 1, 1) KernelSize: (3, 3, 3) linear
batch_normalization_14 (8, 64, 64, 64) -> (8, 64, 64, 64) Strides: None KernelSize: None None
leaky_re_lu_13 (8, 64, 64, 64) -> (8, 64, 64, 64) Strides: None KernelSize: None None
max_pooling3d_6 (8, 64, 64, 64) -> (8, 32, 32, 32) Strides: (2, 2, 2) KernelSize: None None
average_pooling3d_6 (1, 64, 64, 64) -> (1, 32, 32, 32) Strides: (2, 2, 2) KernelSize: None None
merge_6 [(None, 1, 32, 32, 32)] -> (9, 32, 32, 32) Strides: None KernelSize: None None
conv3d_9 (9, 32, 32, 32) -> (24, 32, 32, 32) Strides: (1, 1, 1) KernelSize: (3, 3, 3) linear
batch_normalization_15 (24, 32, 32, 32) -> (24, 32, 32, 32) Strides: None KernelSize: None None
leaky_re_lu_14 (24, 32, 32, 32) -> (24, 32, 32, 32) Strides: None KernelSize: None None
max_pooling3d_7 (24, 32, 32, 32) -> (24, 16, 16, 16) Strides: (2, 2, 2) KernelSize: None None
ave

In [21]:
i.get_config()

{'batch_input_shape': (None, 1, 64, 64, 64),
 'dtype': 'float32',
 'name': 'input_4',
 'sparse': False}